#### Load modules

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

#### Define helper function

In [2]:
def encode(df, column):
    # Get the instance of global dictionary of encoders.
    global encoders
    # Encode given column data.
    if column in encoders:
        return encoders[column].transform(df[column])
    else:
        encoders[column] = LabelEncoder()
        return encoders[column].fit_transform(df[column])

def decode(df, column):
    # Get the instance of global dictionary of encoders.
    global encoders
    # Decode given column data.
    if column in encoder:
        return encoders[column].inverse_transform(df[column])
    else:
        raise "Can't find an appropriate decoder instance."
        
encoders = {}

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    
    return df

#### Preprocess

(1) sales_train_validation.csv

In [4]:
# Load files.
df_sales = pd.read_csv('./sales_train_validation.csv')

# Encode categorical features.
for column in ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']:
    df_sales[column] = encode(df_sales, column)
    
# Unpivot 'd' data.
id_vars = [column for column in df_sales if 'd_' not in column]
value_vars = [column for column in df_sales if 'd_' in column]
df_sales = df_sales.melt(id_vars=id_vars, value_vars=value_vars, var_name='d', value_name='sales')

# Remove 'd_' from 'd' column.
df_sales['d'] = df_sales['d'].apply(lambda x: x.replace('d_', '')).astype('int')

# Reduce memory usage.
df_sales = reduce_mem_usage(df_sales)
df_sales

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,14370,1437,3,1,0,0,1,0
1,14380,1438,3,1,0,0,1,0
2,14390,1439,3,1,0,0,1,0
3,14400,1440,3,1,0,0,1,0
4,14410,1441,3,1,0,0,1,0
...,...,...,...,...,...,...,...,...
58327365,14329,1432,2,0,9,2,1913,1
58327366,14339,1433,2,0,9,2,1913,0
58327367,14349,1434,2,0,9,2,1913,0
58327368,14359,1435,2,0,9,2,1913,3


(2) calendar.csv

In [5]:
# Load files.
df_calendar = pd.read_csv('./calendar.csv')

# Create 'day' column.
df_calendar['day'] = pd.DatetimeIndex(df_calendar['date']).day

# Drop redundant columns.
df_calendar.drop(columns=['date', 'weekday'], inplace=True)

# Encode categorical features.
df_calendar = df_calendar.fillna('')
for column in ['wm_yr_wk', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']:
    df_calendar[column] = encode(df_calendar, column)

# Remove 'd_' from 'd' column.
df_calendar['d'] = df_calendar['d'].apply(lambda x: x.replace('d_', '')).astype('int')

# Reduce memory usage.
df_calendar = reduce_mem_usage(df_calendar)
df_calendar

,wm_yr_wk,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,day
0,0,1,1,2011,1,0,0,0,0,0,0,0,29
1,0,2,1,2011,2,0,0,0,0,0,0,0,30
2,0,3,1,2011,3,0,0,0,0,0,0,0,31
3,0,4,2,2011,4,0,0,0,0,1,1,0,1
4,0,5,2,2011,5,0,0,0,0,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,280,5,6,2016,1965,0,0,0,0,0,1,1,15
1965,280,6,6,2016,1966,0,0,0,0,0,0,0,16
1966,280,7,6,2016,1967,0,0,0,0,0,0,0,17
1967,281,1,6,2016,1968,0,0,0,0,0,0,0,18


(3) sell_prices.csv

In [6]:
# Load files.
df_prices = pd.read_csv('./sell_prices.csv')

# Encode categorical features.
for column in ['store_id', 'item_id', 'wm_yr_wk']:
    df_prices[column] = encode(df_prices, column)
    
# Reduce memory usage.
df_prices = reduce_mem_usage(df_prices)
df_prices

,store_id,item_id,wm_yr_wk,sell_price
0,0,1437,128,9.578125
1,0,1437,129,9.578125
2,0,1437,130,8.257812
3,0,1437,131,8.257812
4,0,1437,132,8.257812
...,...,...,...,...
6841116,9,1436,277,1.000000
6841117,9,1436,278,1.000000
6841118,9,1436,279,1.000000
6841119,9,1436,280,1.000000


(4) merge all

In [7]:
df = pd.merge(df_sales, df_calendar, how='left', on='d')
df = pd.merge(df, df_prices, how='left', on=['store_id', 'item_id', 'wm_yr_wk'])
del df_sales, df_calendar, df_prices

In [8]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,wday,...,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,day,sell_price
0,14370,1437,3,1,0,0,1,0,0,1,...,2011,0,0,0,0,0,0,0,29,NaN
1,14380,1438,3,1,0,0,1,0,0,1,...,2011,0,0,0,0,0,0,0,29,NaN
2,14390,1439,3,1,0,0,1,0,0,1,...,2011,0,0,0,0,0,0,0,29,NaN
3,14400,1440,3,1,0,0,1,0,0,1,...,2011,0,0,0,0,0,0,0,29,NaN
4,14410,1441,3,1,0,0,1,0,0,1,...,2011,0,0,0,0,0,0,0,29,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58327365,14329,1432,2,0,9,2,1913,1,273,2,...,2016,0,0,0,0,0,0,0,24,2.980469
58327366,14339,1433,2,0,9,2,1913,0,273,2,...,2016,0,0,0,0,0,0,0,24,2.480469
58327367,14349,1434,2,0,9,2,1913,0,273,2,...,2016,0,0,0,0,0,0,0,24,3.980469
58327368,14359,1435,2,0,9,2,1913,3,273,2,...,2016,0,0,0,0,0,0,0,24,1.280273
